In [30]:
import os
import sys
import shutil

import pandas as pd
import numpy as np

from geopy import distance

In [31]:
BASE_DIR = os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath(''))))
DB_DIR = os.path.join(BASE_DIR, 'code/hydat_db')
PROJECT_DIR = os.path.abspath('')
IMG_DIR = os.path.join(PROJECT_DIR, 'data/radar_img')
NEW_IMG_DIR = os.path.join(PROJECT_DIR, 'data/sorted_radar_images')

In [32]:
stations = os.listdir(IMG_DIR)

In [33]:
# define radar sites
radar_sites = {'CASAG': {'lat_lon': [49.0580516, -122.470667], # radar location code, lat/lon
                       'scale': 1,
                      'alt_name': 'Aldergrove',
                        }, # km/pixel                       
               'CASPG': {'lat_lon': [53.916943, -122.749443], # radar location code, lat/lon
                       'scale': 1,
                      'alt_name': 'Prince George',}, # km/pixel}, # km/pixel
               'CASSS': {'lat_lon': [50.271790, -119.276505], # radar location code, lat/lon
                       'scale': 1,
                      'alt_name': 'Silver Star',}, # km/pixel}, # km/pixel
               'CASSI': {'lat_lon': [48.407326, -123.329773], # radar location code, lat/lon
                       'scale': 1,
                      'alt_name': 'Victoria',}, # km/pixel}, # km/pixel
               'CASSM': {'lat_lon': [51.206092, -113.399426],
                        'scale': 1,
                        'alt_name': 'Strathmore'},
              }

def find_closest_radar_stn(row):
    """
    To retrieve radar images, we need to find the closest radar location
    to the station of interest.  
    Input the station number,
    returns the location code of the nearest radar station.
    """
    stn_data = row['Station Number']
    
    s1 = (row['Latitude'], row['Longitude'])
    min_dist = 1E6
    closest_stn = None
    for site in radar_sites.keys():

        s2 = [*radar_sites[site]['lat_lon']]        

        this_dist = distance.distance(s2, s1).km
    
        if this_dist < min_dist:
            min_dist = this_dist
            closest_stn = site
        
    return closest_stn

def calc_distance(row):
    wsc_stn_coords = (row['Latitude'], row['Longitude'])
    radar = row['Closest_radar']
    radar_coords = radar_sites[radar]['lat_lon']
    return distance.distance(radar_coords, wsc_stn_coords).km

In [29]:
stations_df = pd.read_csv(DB_DIR + '/WSC_Stations_Master.csv')
stations_df['Closest_radar'] = stations_df.apply(lambda row: find_closest_radar_stn(row), axis=1)
stations_df['Dist_to_radar'] = stations_df.apply(lambda row: calc_distance(row), axis=1)

In [40]:

for stn in stations:
    stn_info = stations_df[stations_df['Station Number'] == stn]
    closest_radar = stn_info['Closest_radar'].values[0]
    save_path = os.path.join(NEW_IMG_DIR, closest_radar)
    
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    existing_files = os.listdir(save_path)
    for r_img in os.listdir(os.path.join(IMG_DIR, stn)):
        if r_img not in existing_files:
            old_img_path = os.path.join(IMG_DIR, stn + '/' + r_img)
            new_img_path = os.path.join(save_path, r_img)

            shutil.copy(old_img_path, new_img_path)
    